In [0]:
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
import pyspark
print(pyspark.__version__)

2.4.5


In [6]:
from google.colab import files
file = files.upload()

Saving customer_churn.csv to customer_churn.csv


In [7]:
!ls

customer_churn.csv  spark-2.4.5-bin-hadoop2.7
sample_data	    spark-2.4.5-bin-hadoop2.7.tgz


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ship').getOrCreate()

In [0]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [10]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [30]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [11]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [12]:
df.head(5)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1),
 Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date=datetime.datetime(2016, 6, 29, 6, 20, 7), Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1),
 Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date=datetime.datetime(2014, 4, 22, 12, 43, 12), Location='13120 Daniel Mount Angelabury, WY 30645-4695'

In [15]:
from pyspark.ml.feature import VectorAssembler

df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], 
                            outputCol='features')

output = assembler.transform(df)

In [17]:
output.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'features']

In [18]:
# Now all i care about is the features & the labels (Churn in this case)
final_data = output.select('features', 'Churn')
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [0]:
# Now split this into training & test data
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                647|
|   mean|0.17001545595054096|
| stddev|0.37593701271840957|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                253|
|   mean|0.15810276679841898|
| stddev| 0.3655605031462895|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [0]:
# Now time for logistic regression
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='features', labelCol='Churn')

r = lr.fit(train_data)

In [29]:
training_summary = r.summary

training_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                647|                647|
|   mean|0.17001545595054096|0.13292117465224113|
| stddev|0.37593701271840957|0.33975218381715394|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [33]:
test_results = r.evaluate(test_data)
test_results.predictions.show()

# prediction is our predicted label

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.48551465999627...|[0.61904923310144...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.32025752967180...|[0.99820375228483...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.24722021410769...|[0.99476541935126...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.72252197300289...|[0.99955745332445...|       0.0|
|[28.0,11204.23,0....|    0|[2.01259199747653...|[0.88211283052707...|       0.0|
|[29.0,10203.18,1....|    0|[3.64824849484076...|[0.97462401446082...|       0.0|
|[29.0,11274.46,1....|    0|[4.50572237361906...|[0.98907506431152...|       0.0|
|[29.0,12711.15,0....|    0|[5.46495229328069...|[0.99578529595299...|       0.0|
|[29.0,13255.05,1....|    0|[4.25094927217350...|[0.98594952934434...|       0.0|
|[30.0,8403.78,1

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

eval_ = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')
area_under_curve = eval_.evaluate(test_results.predictions)

In [39]:
area_under_curve

0.7390258215962441